In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.saving import register_keras_serializable
import tensorflow.keras.backend as K

TRAIN_DATA_DIRECTORY = 'path/to/your/train/data'

In [ ]:
#define HPS
epoch=30
batchsize=32
units1=50
units2=50
lrate=0.001

In [ ]:
#define metrics
# Decorate the custom rmse function
@register_keras_serializable()
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

def r_squared(y_true, y_pred):
    ss_res = K.sum(K.square(y_true - y_pred))
    ss_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - ss_res/(ss_tot + K.epsilon()))

In [ ]:
#read in training data
#read in numpy arrays 
X_train=np.load(f"{TRAIN_DATA_DIRECTORY}/Processed_TF_LSTM/x_train.npy")
X_test=np.load(f"{TRAIN_DATA_DIRECTORY}/Processed_TF_LSTM/x_test.npy")
y_train=np.load(f"{TRAIN_DATA_DIRECTORY}/Processed_TF_LSTM/y_train.npy")
y_test=np.load(f"{TRAIN_DATA_DIRECTORY}/Processed_TF_LSTM/y_test.npy")

In [ ]:
#Define model
model = Sequential()
model.add(LSTM(units1, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units2, return_sequences=False))  # Second LSTM layer with 100 units, return_sequences is False for the final LSTM layer
model.add(Dense(1))
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')


In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=epoch, batch_size=batchsize, verbose=2)
# Create a DataFrame from the training history
history_df = pd.DataFrame(history.history)
history_df['epoch'] = history_df.index + 1  # Add epoch column

# Display the DataFrame
print(history_df)

In [ ]:
#save history as df
history_df.to_csv('history_df_TEST_v1.csv', index=False)

In [ ]:
# Predict and calc metrics  
y_pred = model.predict(X_test)
print(y_pred)
#calculate rmse
rmse_value = np.sqrt(mean_squared_error(y_test,y_pred))
#calculate r2
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {rmse_value}')
print(f'R^2 Score: {r2}')

In [ ]:
#pull out humidity, temp, org purple air for prediction visulizaiton 
X_test

In [ ]:
#record model parameters and metrics 
#save metrics
data = {'epoch': epoch, 
        'batch':batchsize,
        'units1': units1,
        'units2':units2,
        'lrate':lrate,
        'layers':[2],
        'rmse': [rmse_value],
        'r2': [r2]}

df = pd.DataFrame(data)

# Save to CSV
df.to_csv('metrics.csv', index=False)

In [ ]:
#write predictions to csv
df = pd.DataFrame({'prediction_python': y_pred.flatten(), 'y_test_column': y_test.flatten()})

print(df)

df.to_csv('pred_python_base_LSTM.csv')

In [ ]:
#save model
model.save("model_save.keras")